# Retail Pricing Information Distribution Prototype

### Paul M. Washburn

This is a first attempt at developing a prototype to automate the process of reporting current and future pricing to retail accounts.  While the overall process is similar across retailers, each one has a unique set of preferences, file types and information that they supply and demand.  

QuikTrip is the first we will inquire with.  

The goal is to build object-oriented data structures that enable easy processing of inputs and easy generation of outputs in order to save thousands of man-hours that are currently dedicated to this task, thus freeing them from playing defense to playing offense for the firm.  

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
from collections import OrderedDict
import warnings
from matplotlib import pyplot as plt
from pandas.tseries.offsets import YearEnd
from pandas.tseries.holiday import USFederalHolidayCalendar
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 1000)

'''
NOTES TO DEVELOPER:
------------------------------------------------------

'''

# `Product` Objects to Represent Product Elements

The `Product` is the most important object in this process.  The logic calls for many product attributes, then some that it inherits from the (yet-to-be-defined) `ProductPricing` class.  `ProductPricing` will be implemented from the AS400's Frontline report.  

## Read in Raw Data from PRD1 in AS400

In [ ]:
def fetch_pw_prd1(path):
    '''
    Fetches product attributes for merging in later
    '''
    df_prd = pd.read_csv(path + 'pw_prd1.csv', encoding='latin1')
    
    # rename columns
    new_cols = {'PPROD#': 'product_id', 'PDESC': 'description', 'PSIZE@': 'size_code', 'PCLAS@': 'class_code', 
                'PQTYPC': 'qpc', 'PSUPPL': 'supplier_id', 'PBRAN#': 'brand_id', 'PWHSEU': 'warehouse_usage_code', 
                'UPC122': 'upc_code_12_2', 'PUPC': 'upc_code', 'PUPC12': 'upc_code_12_1', 'PUPCS3': 'upc_code_subfield',
                'PSQTYS': 'smallest_qty_sold', 'PUPC2': 'upc_code_2', 'UPC123': 'upc_code_12_3', 'PTYPE': 'type_code', 
                'PSIZE': 'size', 'PSIZE#': 'size_code_redef',  'PPROF': 'proof','PWINTP': 'wine_type', 
                'PCOSEL': 'co_sell_codes', 'PFOB': 'fob_cost_case', 'PPALSZ': 'pallet_size', 'PTIE': 'ties',
                'PTIER': 'tiers', 'PBPBKC': 'price_book_code', 'PBDESC': 'price_book_description', 'PVINTG': 'vintage',
                'PCOLR': 'color', 'PGIFT': 'gift_set', 'PCOPAK': 'co_pack'}
    df_prd.rename(columns=new_cols, inplace=True)
    
    return df_prd

DATASETS = {}

# read in products
df_all_products = fetch_pw_prd1(path='C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Generalized Lookup Data\\')
DATASETS['products_raw'] = df_all_products

print(DATASETS['products_raw'].head())

## Creating the `Product` Object

The `Product` object is one of the main objects in any solution architecture to this problem.  Certain base attributes and methods must be instantiated before more complex inheritances may occur.  

In [ ]:
class Product(object):
    def __init__(self, product_id, description, size_code, qpc, supplier_id, 
                 brand_id, class_code, type_code, upc_permutations,
                 warehouses_carried, co_sell_codes, price_book_code, private_label,
                 seasonality, cont_type, pallet_dims, proof, vintage, color, gift_set):
        '''
        Product represents a unique Product in our portfolio. All data needed can be obtained 
        by running the AS400 query pw_prd1.
        
        Instantiate with:
        --------------------------------------------------
         product_id:
         description:
         size_code:
         qpc:
         supplier_id: will be replaced with object in future
         brand_id:
         class_code:
         type_code: 
         upc_permutations:
         warehouses_carried:
         co_sell_codes: 
         price_book_code:
         private_label:
         seasonality: {'seas_code': val, 'seas_desc': val}
         cont_type:
         pallet_dims: {'pallet_size': val, ties': val, 'tiers': val}
         proof: 
         vintage:
         color:
         gift_set:
        '''
        self.product_id = product_id
        self.description = description
        self.size_code = size_code
        self.qpc = qpc
        self.supplier_id = supplier_id
        self.brand_id = brand_id
        self.class_code = class_code
        self.type_code = type_code
        self.upc_permutations = upc_permutations
        self.warehouses_carried = warehouses_carried
        self.co_sell_codes = {'kc': str(co_sell_codes)[0]=='1',
                              'stl': str(co_sell_codes)[1]=='1',
                              'col': str(co_sell_codes)[2]=='1',
                              'cape': str(co_sell_codes)[3]=='1',
                              'spfd': str(co_sell_codes)[4]=='1'}
        self.price_book_code = price_book_code
        self.private_label = private_label
        self.seasonality = seasonality
        self.cont_type = cont_type
        self.pallet_dims = pallet_dims
        
    def __str__(self):
        return '''
        Product:                    %s  -  %s
        Supplier:                   %s
        Company Sell Codes          %s
        ''' %(self.product_id, self.description, 
              self.supplier_id,
              self.co_sell_codes)
        
    def make_private_label(self):
        self.private_label = True
        
    def append_upcs(self, new_upc_dict):
        self.upc_permutations.update(new_upc_dict)

## Instantiate `Product` Objects and Save to `PRODUCT_MASTER` `dict` Object for Use

In [ ]:
# get columns for later
upc_columns = [col for col in DATASETS['products_raw'].columns if 'upc' in col]
pallet_columns = ['pallet_size', 'ties', 'tiers']

# instantiate Product objects in list
PRODUCT_MASTER = {}
for i, product_id in enumerate(DATASETS['products_raw']['product_id'].tolist()): #already unique bc from PRD1
    # extract all needed dicts from data
    upc_permutations = DATASETS['products_raw'].loc[i, upc_columns].to_dict()
    pallet_dims = DATASETS['products_raw'].loc[i, pallet_columns].to_dict()
    
    # derive seasonality
    seasonality = {} #placeholder -- WHERE TO GET?  may need a setter method to set from frontline
    
    # cont_type same as above
    cont_type = None
    
    # mark private labels placeholder
    private_label = False
    
    # get companies sold at 
    warehouses_carried = []
    
    # actually instantiate product and save to PRODUCT_MASTER
    PRODUCT_MASTER[product_id] = Product(product_id, 
                                         DATASETS['products_raw'].loc[i, 'description'],
                                         DATASETS['products_raw'].loc[i, 'size_code'],
                                         DATASETS['products_raw'].loc[i, 'qpc'], 
                                         DATASETS['products_raw'].loc[i, 'supplier_id'],
                                         DATASETS['products_raw'].loc[i, 'brand_id'],
                                         DATASETS['products_raw'].loc[i, 'class_code'],
                                         DATASETS['products_raw'].loc[i, 'type_code'],
                                         upc_permutations,
                                         warehouses_carried, #ask holly how to parse
                                         DATASETS['products_raw'].loc[i, 'co_sell_codes'],
                                         DATASETS['products_raw'].loc[i, 'price_book_code'],
                                         private_label,
                                         seasonality,
                                         cont_type,
                                         pallet_dims,
                                         DATASETS['products_raw'].loc[i, 'proof'],
                                         DATASETS['products_raw'].loc[i, 'vintage'],
                                         DATASETS['products_raw'].loc[i, 'color'],
                                         DATASETS['products_raw'].loc[i, 'gift_set'])

In [ ]:
print(PRODUCT_MASTER[31125])

## `ProductPricing` Class to Represent Frontline Pricing at a Given Point in Time

The AS400 Frontline report is used as input for this class.

### Read in Frontline Data

In [ ]:
as400_date = lambda dat: dt.strptime(str(dat)[-6:], '%y%m%d')

class AS400FrontLineReport(object):
    def __init__(self, path):
        self.data = self.read_and_process_frontline(path)
        self.run_date = dt.now()
        
    def __getattr__(self, attr):
        if hasattr(self, attr):
            return self.attr

    def read_and_process_frontline(self, path):
        # read in frontline query
        df = pd.read_csv(path + 'FRONTLNST.csv', header=0, encoding='latin-1')

        # replace blank space in column names
        df.columns = [str(col).replace(' ', '_') for col in df.columns]
        df.columns = [str(col).replace('#', 'ID') for col in df.columns]
        df.columns = [str(col).lower() for col in df.columns]

        # process specific columns
        df['effective_date'] = df['effective_date'].apply(as400_date)
        
        # rename columns to match
        dis = ['upcinner', 'upcinner_type', 'upccase', 'upccase_type', 'upcbshort', 'upcishort', 'upccshort', 'upcbttl', 'upcbttl_type']
        dat = ['upc_inner', 'upc_inner_type', 'upc_case', 'upc_case_type', 'upc_b_short', 'upc_i_short', 'upc_c_short', 'upc_btl', 'upc_btl_type']
        new_names = dict(zip(dis, dat))
        df.rename(columns=new_names, inplace=True)
        
        dis = ['sell_prc_1', 'nm_sell_prc_1', 'nm_+_1_sell_prc_1', 'nm_+_2_sell_prc_1', 'sell_prc_2', 'nm_sell_prc_2', 'nm_+_1_sell_prc_2', 'nm_+_2_sell_prc_2']
        dat = ['prc1_month_0', 'prc1_month_1', 'prc1_month_2', 'prc1_month_3', 'prc2_month_0', 'prc2_month_1', 'prc2_month_2', 'prc2_month_3']
        new_names = dict(zip(dis, dat))
        df.rename(columns=new_names, inplace=True)
        
        # get rid of decimals not necessary
        df[dat] = df[dat].astype(np.float32).apply(lambda x: round(x, 3))
        
        return df

# set file path
path = 'C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Automation\\Retail Pricing\\Data\\'

# instantiate frontline query
frontline = AS400FrontLineReport(path)

# cache data in DATASETS dictionary for later access
DATASETS['frontline'] = frontline.data
DATASETS['frontline'].loc[DATASETS['frontline']['product_id']==14012]

In [ ]:
# print column names
print(DATASETS['frontline'].columns)

# demonstrate that both point to same object, frontline.data
# i.e. we are not duplicating disk space storage, always point back
# to the original object

DATASETS['frontline'] is frontline.data

## Create `ProductPricing` Class

This is the most important class in the architecture. 

In [ ]:
# instantiate classes not yet fleshed out

class MajorBrandsCompany:
    def __init__(self, company_id):
        self.company_id = company_id
        
class Director:
    def __init__(self, director_id, director_name):
        self.director_id = director_id
        self.director_name = director_name
        
class Manager:
    def __init__(self, manager_id):
        self.manager_id = manager_id
        
class Division:
    def __init__(self, division_id):
        self.division_id = division_id
        
class ReportingGroup:
    def __init__(self, rpt_group_id):
        self.rpt_group_id = rpt_group_id
        
class Supplier:
    def __init__(self, supplier_id, supplier_name):
        self.supplier_id = supplier_id
        self.supplier_name = supplier_name
        
        
class ProductPricing(Product):
    def __init__(self, effective_date, Product, smallest_qty, retail_sell_info, base_prices, 
                 supplier_list_prices, ReportingGroup, available_cases, on_order, 
                 MajorBrandsCompany, Division, priv_label, season_info, cont_type,
                 sell_price_1, sell_price_2, price_group_code, vap, discounts, wine_group_info):
        '''
        ProductPricing extends the Product class by making a new object which inherits all of the 
        characteristics of Product and in addition has information about the next three months of 
        pricing for the Product in question.  
        
        All information comes from the AS400 Frontline Pricing Report.
        
        Instantiate with:
        --------------------------------------------------
         effective_date: 
         Product: base Product class
         smallest_qty: 
         retail_sell_info: dict of ['rtl_sell_unit', 'rtl_sell_pack'] 
         base_prices: dict of ['base_btl_price', 'base_case_price']
         supplier_list_prices: dict of ['sup_list_case', 'sup_list_btl'] 
         ReportingGroup: 
         available_cases: 
         on_order: 
         MajorBrandsCompany: 
         Division: 
         priv_label: 
         season_info: 
         cont_type:
         sell_price_1: dict of ['sell_prc_1', 'nm_sell_prc_1', 'nm_+_1_sell_prc_1', 'nm_+_2_sell_prc_1']
         sell_price_2: dict of ['sell_prc_2', 'nm_sell_prc_2', 'nm_+_1_sell_prc_2', 'nm_+_2_sell_prc_2']
         price_group_code: 
         vap: 
         discounts: dict of ['disco_kc', 'disco_stl']
         wine_group_info: dict of ['wine_desc', 'wine_comment', 'wine_rpt_grp_type', 'wine_rpt_grp_title']
        '''
        self.effective_date = effective_date
        self.product = Product
        self.smallest_qty = smallest_qty
        self.retail_sell_info = retail_sell_info
        self.base_prices = base_prices
        self.supplier_list_prices = supplier_list_prices
        self.reporting_group = ReportingGroup
        self.available_cases = available_cases
        self.on_order = on_order
        self.company = MajorBrandsCompany
        self.division = Division
        self.priv_label = priv_label
        self.season_info = season_info
        self.cont_type = cont_type
        self.sell_price_1 = sell_price_1
        self.sell_price_2 = sell_price_2
        self.price_group_code = price_group_code
        self.vap = vap
        self.discounts = discounts
        self.wine_group_info = wine_group_info
        
    def __str__(self):
        return '''
        ProductPricing object:
        ----------------------------------------------------------------
        Product:                    %s  -  %s
        
        Sell Price 1:               
        %s
        
        Sell Price 2:               
        %s
        
        Sell Codes:                 
        %s
        ''' %(self.product.product_id, self.product.description,
             pd.Series(self.sell_price_1), pd.Series(self.sell_price_2),
             pd.Series(self.product.co_sell_codes))

## Instantiate `ProductPricing` Objects and Save to `PRODUCT_PRICING_MASTER` List

In [ ]:
# cache columns for all composite attributes of ProductPricing objects
frontline_upc_cols = ['upc_inner', 'upc_inner_type', 'upc_case', 'upc_case_type', 'upc_b_short', 'upc_i_short', 
                      'upc_c_short', 'upc_btl', 'upc_btl_type']
wine_grp_cols = ['wine_desc', 'wine_comment', 'wine_rpt_grp_type', 'wine_rpt_grp_title']
sell_price_1_cols =['prc1_month_0', 'prc1_month_1', 'prc1_month_2', 'prc1_month_3']
sell_price_2_cols = ['prc2_month_0', 'prc2_month_1', 'prc2_month_2', 'prc2_month_3']
season_info_cols = ['seas_code', 'seas_desc', 'seas_start']
base_price_cols = ['base_btl_price', 'base_case_price']
retail_sell_info_cols = ['rtl_sell_unit', 'rtl_sell_pack']
discount_cols = ['disco_kc', 'disco_stl']
supplier_list_cols = ['sup_list_case', 'sup_list_btl'] 

# create master dictionaries to avoid redundant memory allocation
PRODUCT_PRICING_MASTER, REPORTING_GROUP_MASTER, MB_COMPANY_MASTER, DIVISION_MASTER = {}, {}, {}, {}

df = DATASETS['frontline'].copy()
for i, pid in enumerate(df.product_id):

    # update upc list on Product objects before instantiating ProductPricing objects
    upc_frontline_pid = df.loc[i, frontline_upc_cols].to_dict()
    PRODUCT_MASTER[pid].append_upcs(upc_frontline_pid)

    # save column info from central dataset for instantiation
    wine_group_info = df.loc[i, wine_grp_cols].to_dict()
    sell_price_1 = df.loc[i, sell_price_1_cols].to_dict()
    sell_price_2 = df.loc[i, sell_price_2_cols].to_dict()
    season_info = df.loc[i, season_info_cols].to_dict()
    base_prices = df.loc[i, base_price_cols].to_dict()
    retail_sell_info = df.loc[i, retail_sell_info_cols].to_dict()
    discounts = df.loc[i, discount_cols].to_dict()
    supplier_list_prices = df.loc[i, supplier_list_cols].to_dict()

    # if they haven't been yet, instantiate satellite objects ReportingGroup, MajorBrandsCompany & Division
    # these objects currently have limited functionality, this enables extensability later
    if df.loc[i, 'rpt_group'] not in REPORTING_GROUP_MASTER.keys():
        reporting_group_obj = ReportingGroup(df.loc[i, 'rpt_group'])
        REPORTING_GROUP_MASTER[df.loc[i, 'rpt_group']] = reporting_group_obj

    if df.loc[i, 'company'] not in MB_COMPANY_MASTER.keys():
        mb_company_group_obj = MajorBrandsCompany(df.loc[i, 'company'])
        MB_COMPANY_MASTER[df.loc[i, 'company']] = mb_company_group_obj

    if df.loc[i, 'division'] not in DIVISION_MASTER.keys():
        division_obj = Division(df.loc[i, 'division'])
        DIVISION_MASTER[df.loc[i, 'division']] = division_obj

    ##FINAL STEP
    # instantiate ProductPricing objects
    product_pricing_obj = ProductPricing(df.loc[i, 'effective_date'],
                                        PRODUCT_MASTER[pid], 
                                        df.loc[i, 'smallest_qty'],
                                        retail_sell_info,
                                        base_prices,
                                        supplier_list_prices,
                                        reporting_group_obj,
                                        df.loc[i, 'available_cases'],
                                        df.loc[i, 'on_order'],
                                        MB_COMPANY_MASTER[df.loc[i, 'company']],
                                        DIVISION_MASTER[df.loc[i, 'division']],
                                        df.loc[i, 'priv_label'],
                                        season_info,
                                        df.loc[i, 'cont_type'],
                                        sell_price_1,
                                        sell_price_2,
                                        df.loc[i, 'price_grp_code'],
                                        df.loc[i, 'vap_y/n'],
                                        discounts,
                                        wine_group_info)
    
    # cache by product id
    PRODUCT_PRICING_MASTER[pid] = product_pricing_obj
    
del df

In [ ]:
print(PRODUCT_PRICING_MASTER[14012])

# Specify `ChainStore` & `RetailCustomer` Classes

In [ ]:
def fetch_customer_product_dive(path):
    '''
    Fetches Diver Marker for customer-product purchases.  Data source
    must be updated prior to running if new information is desired. 
    '''
    customer_product_dive_df = pd.read_csv(path + 'RETAIL PRICING -- Customer Dive.csv',
                                          encoding='utf8')
    
    # replace blank space in column names
    customer_product_dive_df.columns = [str(col).replace(' ', '_') for col in customer_product_dive_df.columns]
    customer_product_dive_df.columns = [str(col).lower() for col in customer_product_dive_df.columns]
    
    # get on-premise True or False
    customer_product_dive_df['on_premise'] = customer_product_dive_df['premise_type'] != 'Off-Premise'
    
    # fill na
    customer_product_dive_df['chain_name'] = customer_product_dive_df['chain_name'].fillna('')
    
    return customer_product_dive_df


class ChainStore(object):
    def __init__(self, chain_id, chain_name):
        self.chain_id = chain_id
        self.chain_name = chain_name
        self.retail_customers = []
        
    def __str__(self):
        return '''
        Chain Store:       %s  -  %s
        ''' %(self.chain_id, self.chain_name)
    
    def append_customer(self, RetailCustomer):
        self.retail_customers.append(RetailCustomer)
        
        
class RetailCustomer(ChainStore):
    def __init__(self, customer_id, customer_name, ChainStore, MajorBrandsCompany, warehouse, output_preferences,
                frequency_preferences, on_premise):
        '''
        RetailCustomer
        
        Instantiate with:
        --------------------------------------------------
         customer_id:
         customer_name:
         ChainStore: ChainStore object (max of 1)
         MajorBrandsCompany: MajorBrandsCompany object (max of 1)
         output_preferences:
         frequency_preferences:
         on_premise: True if on-premise, o.w. False
        '''
        self.customer_id = customer_id
        self.customer_name = customer_name
        self.chain_store = ChainStore
        self.company = MajorBrandsCompany
        self.warehouse = warehouse
        self.output_preferences = output_preferences
        self.frequency_preferences = frequency_preferences
        self.on_premise = on_premise
        self.products_carried = []
        self.number_products_carried = len(self.products_carried)
        
    def __str__(self):
        return '''
        Customer:           %s  -  %s
        Warehouse:          %s
        On Premise:         %s
        
        Chain Affiliate:    %s
        ''' %(self.customer_id, self.customer_name, self.warehouse, self.on_premise, self.chain_store)
    
    def append_product(self, Product):
        self.products_carried.append(Product)
        self.number_products_carried = len(self.products_carried)

### Instantiate `ChainStore` & `RetailCustomer` Class Objects

In [ ]:
# read in Diver marker that contains chain/customer and products carried data
tmp_path = 'C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Generalized Lookup Data\\'
custprod_dive_df = fetch_customer_product_dive(tmp_path)


# accumulate all chain stores into a dictionary
CHAIN_STORE_MASTER = {}
for ix, row in custprod_dive_df.loc[custprod_dive_df.chainid != 0, ['chainid', 'chain_name']].drop_duplicates().iterrows():
    CHAIN_STORE_MASTER[row['chainid']] = ChainStore(row['chainid'], row['chain_name'])
    
    
# accumulate all customer objects into a dictionary
RETAIL_CUSTOMER_MASTER = {}
for ix, row in custprod_dive_df[['customer_id', 'customer', 'on_premise', 'chainid', 'warehouse']].drop_duplicates().iterrows():
    
    if row['chainid'] != 0:
        this_chain = CHAIN_STORE_MASTER[row['chainid']]
    else:
        this_chain = None
    
    customer_obj = RetailCustomer(customer_id=row['customer_id'], 
                                  customer_name=row['customer'], 
                                  ChainStore=this_chain, 
                                  MajorBrandsCompany=None,
                                  warehouse=row['warehouse'],
                                  output_preferences={},
                                  frequency_preferences={}, 
                                  on_premise=row['on_premise'])
    RETAIL_CUSTOMER_MASTER[row['customer_id']] = customer_obj


print(CHAIN_STORE_MASTER[5188975])    
print(RETAIL_CUSTOMER_MASTER[5188975])

In [ ]:
custprod_dive_df.tail(40)

In [ ]:
print(RETAIL_CUSTOMER_MASTER[5900017])

In [ ]:
class CustomerInputData(RetailCustomer):
    def __init__(self, RetailCustomer, file_type, foreign_sku_list, foreign_sku_type, 
                upc_list, upc_type):
        '''
        CustomerInputData
        
        Instantiate with:
        --------------------------------------------------
         RetailCustomer:
         file_type:
         foreign_sku_list:
         foreign_sku_type:
         upc_list: 
         upc_type:
        '''
        
    def __str__(self):
        return '''
        Customer Input Data for %s  -  %s
        ----------------------------------------------------------------
        File type:                        %s
        Number of Products                %s
        ''' %()

# `CustomerInputData` Class

In [ ]:
class RetailChain(object):
    pass

class Customer(RetailChain):
    pass

class CustomerInputData(Customer):
    pass



In [ ]:
DATASETS['products_raw'].loc[1, ['product_id', 'supplier_id']].to_dict()

In [ ]:
DATASETS['products_raw'].loc[599, [col for col in DATASETS['products_raw'].columns if 'upc' in col]].astype(int).to_dict()

In [ ]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from datetime import datetime as dt

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
%matplotlib inline


# as400_date = lambda dat: dt.strptime(str(dat)[-6:], '%y%m%d')

# class AS400FrontLineReport(object):
#     def __init__(self, path):
#         self.data = self.read_and_process_frontline(path)
#         self.run_date = dt.now()
        
#     def __getattr__(self, attr):
#         if hasattr(self, attr):
#             return self.attr

#     def read_and_process_frontline(self, path):
#         # read in frontline query
#         df = pd.read_csv(path + 'FRONTLNST.csv', header=0, encoding='latin-1')

#         # replace blank space in column names
#         df.columns = [str(col).replace(' ', '_') for col in df.columns]
#         df.columns = [str(col).replace('#', 'ID') for col in df.columns]
#         df.columns = [str(col).lower() for col in df.columns]

#         # process specific columns
#         df['effective_date'] = df['effective_date'].apply(as400_date)

#         return df

    
class RetailCustomer(object):
    def __init__(self, *retail_info):    
        for dictionary in retail_info:
            for key in dictionary:
                setattr(self, key, dictionary[key])
    
    def __getattr__(self, attr):
        if hasattr(self, attr):
            return self.attr
        
    def add_products(self, product_id_list):
        self.product_ids = self.product_ids + product_id_list


class GenericRetailProductReport(object):
    def __init__(self, *report_info):    
        for dictionary in report_info:
            for key in dictionary:
                setattr(self, key, dictionary[key])


class ExcelRetailProductReport(GenericRetailProductReport):
    def __init__(self, *report_info, **report_details):
        # parent class attributes
        GenericRetailProductReport.__init__(self, *report_info)
        
        # subclass attributes
        for key in report_details:
            setattr(self, key, report_details[key])
        wb = pd.ExcelFile(self.input_file_path)
        ws = wb.parse(sheet=self.sheet, skiprows=self.skiprows)
        ws = ws[[col for col in ws.columns if 'Unnamed' not in col]]
        self.data = ws
    
    def __getattr__(self, attr):
        if hasattr(self, attr):
            return self.attr
        
    def derive_product_ids(self, external_product_id, internal_product_id):
        product_df = pd.DataFrame({
            external_product_id:self.data[external_product_id].values,
            internal_product_id:self.data[internal_product_id]
        })
        new_col_names = {external_product_id:'external_product_id', internal_product_id:'internal_product_id'}
        product_df.rename(columns=new_col_names, inplace=True)
        product_df = product_df.loc[product_df['external_product_id'].isnull() == False]
        
        return product_df
    
    
def generate_customer_pricing_report(customer_info, customer_report_info, customer_report_details):
    '''
    Employs object classes already created to generate the customer_report
    object, which contains all information needed for downstream processes
    with respect to retail pricing requests.
    
    Inputs
    --------------------------------------
     customer_info: 
     customer_report_info:
     customer_report_details:
    
    Outputs
    --------------------------------------
     customer_report: 
    '''
    # instantiate customer object
    customer = RetailCustomer(fake_customer_info)

    # instantiate customer input data
    customer_report_info.update({'_customer':customer})
    customer_report = ExcelRetailProductReport(customer_report_info, customer_report_details)
    
    # extract customer products carried and add to customer object
    customer_products = customer_report.derive_product_ids(external_product_id=customer_report.external_product_id, 
                                                           internal_product_id=customer_report.internal_product_id)
    customer_products = customer_products['internal_product_id'].dropna().tolist()
    customer.add_products(customer_products)

    return customer_report

In [ ]:
# set file path
path = 'C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Automation\\Retail Pricing\\Data\\'

# instantiate frontline query
frontline = AS400FrontLineReport(path)

## House this stuff in a spreadsheet 
## so people can edit it
## Ensure there are safeguards against bad values

# fake customer example
fake_customer_info = {
    'customer_id':12345, 
    'customer_name':'New Customer # 7',
    'product_ids':[], #placeholder 
    'product_id_type':'UPC', 
    'warehouse':'STL'
}
# fake customer report information
customer_report_info = {
    'input_file_type':'xlsx',
    'uses_crossref_file':True, 
    'group':'D', 
    'case_buy':True, 
    'btl_buy':True, 
    'nine_mo_pricing':False, 
    'sends_file':True,
    'external_product_id':'SKU', 
    'internal_product_id':'Vendor\'s Item No.'
}
# fake customer report details
customer_report_details = {
    'input_file_path':path + 'QT Product Pricing Input.xlsx',
    'sheet':0, 
    'skiprows':5
}

customer_report = generate_customer_pricing_report(fake_customer_info, 
                                                   customer_report_info, 
                                                   customer_report_details)

#customer_products = customer_report.derive_product_ids(sku_col='SKU', mb_productid_col='Vendor\'s Item No.')
customer_report._customer.customer_id

In [ ]:
customer_products['internal_product_id'].tolist()

In [ ]:
print(frontline.data.head(1))

In [ ]:
pd.unique(frontline.data.effective_date)

In [ ]:
import glob
for file in glob.glob(path + '*'):
    #print(file)
    pass

In [ ]:
class Canvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.data = [[' '] * width for i in range(height)]

    def setpixel(self, row, col):
        self.data[row][col] = '*'

    def getpixel(self, row, col):
        return self.data[row][col]

    def display(self):
        print( "\n".join(["".join(row) for row in self.data]))

class Shape:
    def paint(self, canvas): pass

class Rectangle(Shape):
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def hline(self, x, y, w):
        pass

    def vline(self, x, y, h):
        pass

    def paint(self, canvas):
        hline(self.x, self.y, self.w)
        hline(self.x, self.y + self.h, self.w)
        vline(self.x, self.y, self.h)
        vline(self.x + self.w, self.y, self.h)

class Square(Rectangle):
    def __init__(self, x, y, size):
        Rectangle.__init__(self, x, y, size, size)

class CompoundShape(Shape):
    def __init__(self, shapes):
        self.shapes = shapes

    def paint(self, canvas):
        for s in self.shapes:
            s.paint(canvas)
            
            
canvas = Canvas(10, 40)
canvas.display()